# Homework 1 - Streaming and Higher Order Functions (15 pts)

Please turn in your note book naming as *BDM\_HW1\_LastName.ipynb*.


## Task 1 (2 points)

Please complete the **lambda f1** definition below by filling in the _________ part. **f1** is expected to take a single string argument and returns whether the string can be converted to a natural number or zero. If it's possible, returns **'Number'**, otherwise returns **'Not a number'**. For example *'0123'* is a valid number 'Number', whereas *'0xff'* is not.

In [ ]:
f1 = lambda x: 'Number' if (x.isdigit()) else 'Not a number'
print(f1('1A'))
print(f1('12'))
print(f1('b1'))

Not a number
Number
Not a number



## Task 2 (2 points)

Please complete the **lambda f2** definition below by filling in the _________ part. **f2** takes a single iterable (e.g. a list or a string), and returns the number of unique elements. Please see the sample output below.

In [ ]:
f2 = lambda x: len(set(x)) 
print(f2([1,2,3,4,1,5,2]))
print(f2('hello world'))

5
8



## Task 3 (2 points)

Please complete the **lambda f3** definition below by filling in the _________ part. **f3** takes two strings *x* and *y*, and return all the words in *x* that do not appear in *y*. Please note that the word comparison iscase insensitive. Samples area also provided below.

In [ ]:
f3 = lambda x, y: set(x.split( )).difference(set(y.split( )))
print(f3('big data management and analysis', 'big data computing'))
print(f3('this is a phrase', 'this is another phrase'))

{'management', 'analysis', 'and'}
{'a'}



## Task 4 (9 points)

We are greatly inspired by the [Consumer Complaints](https://github.com/InsightDataScience/consumer_complaints) challenge from [InsightDataScience](https://github.com/InsightDataScience/). In fact, we are going to tackle the same challenge but streaming. Please read through the challenge at the following link (the most important sections are “Input dataset” and “Expected output”):

<https://github.com/InsightDataScience/consumer_complaints>

Our problem is different than the one above in that the INPUT is also <ins>sorted by product (alphabetically) and year (ascending)</ins>, similar to the requested output format. You are asked to complete the generator below to produce the expected output. You should not edit any other code.

### INPUT
Your code will be evaluated against the original data set (in CSV format) downloaded from the below link (of course with a sorted version). For your convenience, a smaller version of the file is provided below for testing purposes.

https://www.consumerfinance.gov/data-research/consumer-complaints/#download-the-data



In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
drive.CreateFile({'id': '1-IeoZDwT5wQzBUpsaS5B6vTaP-2ZBkam'}).GetContentFile('complaints.csv')
!head -n 1 complaints.csv

Date received,Product,Sub-product,Issue,Sub-issue,Consumer complaint narrative,Company public response,Company,State,ZIP code,Tags,Consumer consent provided?,Submitted via,Date sent to company,Company response to consumer,Timely response?,Consumer disputed?,Complaint ID


In [70]:
import datetime
def task4(reader):
  # YOU CAN ONLY EDIT CODE BELOW THIS LINE
  # Note: below is just an example, and not correct
  total_complains = 0
  last_product= None
  last_year = None
  sum_comp = 0
  company = {}
  for row in reader: # this line should stay the same
    #day = datetime.datetime.strptime(row['Date received'], '%Y-%m-%d')
    #year = row['Date received'][0:4]
    if ((last_product==row['Product'] and last_year== row['Date received'][0:4]) or (last_product == None and last_year == None)):
      total_complains+=1
      last_product = row['Product']
      last_year = row['Date received'][0:4]
      company[row['Company']]= company.get(row['Company'],0)+1
      sum_comp=len(company.keys())
      #MaxKey = max(companyb , key=companyb.count)
      MaxKey = max(company, key=company.get)
      #Max_bank = max(company.values())
      percentage =round((company[MaxKey]/sum_comp)*100)
      #percentage =round((Max_bank/sum_comp)*100)
    else:

      yield ([last_product.lower(),last_year,total_complains,sum_comp,percentage])
      last_product = row['Product']
      last_year = row['Date received'][0:4]
      total_complains = 1
      company={}
      company[row['Company']]= company.get(row['Company'],0)+1
      sum_comp=len(company.keys())
      MaxKey = max(company, key=company.get)
      percentage =round((company[MaxKey]/sum_comp)*100)
      #last_product = row['Product']
      #last_year = year

In [71]:
# DO NOT EDIT THIS CELL, YOU CAN RUN TO QUICKLY CHECK YOUR RESULTS

import csv

with open('complaints.csv', 'r') as fi, open('output.csv', 'w') as fo:
  reader = csv.DictReader(fi)
  writer = csv.writer(fo)
  for row in task4(reader):
    writer.writerow(row)

!head output.csv

bank account or service,2015,1,1,100
bank account or service,2016,2,2,50
checking or savings account,2017,1,1,100
checking or savings account,2018,20,10,50
checking or savings account,2019,461,72,86
checking or savings account,2020,3,3,33
consumer loan,2015,1,1,100
consumer loan,2016,1,1,100
consumer loan,2017,1,1,100
credit card,2016,4,4,25
